In [3]:
import os
from dotenv import load_dotenv
load_dotenv()

assert os.environ["YA_OBJ_STORAGE_BUCKET"]
assert os.environ["YA_OBJ_STORAGE_AWS_ACCESS_KEY_ID"]
assert os.environ["YA_OBJ_STORAGE_AWS_SECRET_ACCESS_KEY"]
assert os.environ["YA_SPEECH_KIT_API_KEY"]
assert os.environ["YA_SPEECH_KIT_SECRET"]

In [ ]:
import json
import os
from src.audio_convert import aac_to_opus
from src.config import DATA_DIR
from src.file import filename_without_extension_from_path
from src.ya_obj_storage import UploadFileToYaObjCloud
from src.ya_speech_to_text import YaSpeechToTextCli, merge_parsed_text

In [2]:
# Определяем файлы, которые надо распознать

# Если файлы в aac, то их надо в opus перевести
aac_files = [

]
print("Переводим aac в opus...")
opus_files = [
    aac_to_opus(aac)
    for aac in aac_files
]
print("\n".join(opus_files))

# Если файлы уже в opus, то ячейку нуже не нужно запускать
# Нужно создать переменную opus_files
# opus_files = [
# ]




Переводим aac в opus...
C:\Users\GANSOR\PyCharmProjects\yaSpeechToText\data\opus\деплой 2.opus


In [3]:
print("Загружаем opus в yandex-object-cloud...")
opus_urls = [
    UploadFileToYaObjCloud()(opus)
    for opus in opus_files
]
print("\n".join(opus_urls))


Загружаем opus в yandex-object-cloud...
https://storage.yandexcloud.net/ya-speech-to-text/деплой 2.opus


In [4]:
# Создаем клиента
cli = YaSpeechToTextCli()


In [5]:
print("Отправляем запросы на расшифровку в yandex-speech-kit...")
recognize_request_ids = [
    cli.start_recognize_audio(opus)
    for opus in opus_urls
]
print(json.dumps(recognize_request_ids, indent=2))

Отправляем запросы на расшифровку в yandex-speech-kit...
[
  {
    "done": false,
    "id": "e038vvu533oi9n3k3cdp",
    "createdAt": "2021-10-13T17:12:46Z",
    "createdBy": "aje1ii2kmcgldklcpc9p",
    "modifiedAt": "2021-10-13T17:12:46Z"
  }
]


In [7]:
print("Чекаем статус расшифровки...")

recog_responses = [
    cli.get_recognize_status(resp)
    for resp in recognize_request_ids
]
assert all(resp["done"] for resp in
           recog_responses), f"Еще не готово:\n{next(resp for resp in recog_responses if not resp['done'])}"
print("Все ок")

Чекаем статус расшифровки...
Все ок


In [8]:
print("Делаем текст...")
recog_texts = [
    merge_parsed_text(resp)
    for resp in recog_responses
]
recog_files = [
    filename_without_extension_from_path(file) + ".txt"
    for file in opus_files
]
recog_paths = [
    os.path.join(DATA_DIR, "text", file)
    for file in recog_files

]
for path, text in zip(recog_paths, recog_texts):
    with open(path, mode="w", encoding="utf-8") as f:
        f.write(text)

print("\n".join(recog_paths))

Делаем текст...
C:\Users\GANSOR\PyCharmProjects\yaSpeechToText\data\text\деплой 2.txt
